<img src="data/Caption_Picture.jpg">

# Dog Breed Identification 

## Determine the breed of a dog in an image

In this project, we are going to model a <b>Convolution Neural Network</b> to identify the breed of a dog from a given image.

The input will be the <b>RGB</b> values matrix of the pixels in the images, the cnn will then outputs the probability of the dog belonging to each and every breed.

## Data Set

The data set used for the analysis is provided by Stanford Univeristy.

Link for the dataset: http://vision.stanford.edu/aditya86/ImageNetDogs/

#### Data set specifications

* __No of classes__ : 120
* __No of train images__ : 12000 (_Atleast 100 per class_)
* __No of test Images__ : 8550

Randomly chosen pics from the data set are shown below:

<img src="data/sample_images.png">

In [2]:
import pandas as pd

data_set = pd.read_csv("data/data_dict.csv")

In [6]:
count = 1
for breed in data_set["breed_name"].unique():
    print("%d. %s" % (count, breed))
    count += 1

1. Chihuahua
2. Japanese_spaniel
3. Maltese_dog
4. Pekinese
5. Shih-Tzu
6. Blenheim_spaniel
7. papillon
8. toy_terrier
9. Rhodesian_ridgeback
10. Afghan_hound
11. basset
12. beagle
13. bloodhound
14. bluetick
15. black-and-tan_coonhound
16. Walker_hound
17. English_foxhound
18. redbone
19. borzoi
20. Irish_wolfhound
21. Italian_greyhound
22. whippet
23. Ibizan_hound
24. Norwegian_elkhound
25. otterhound
26. Saluki
27. Scottish_deerhound
28. Weimaraner
29. Staffordshire_bullterrier
30. American_Staffordshire_terrier
31. Bedlington_terrier
32. Border_terrier
33. Kerry_blue_terrier
34. Irish_terrier
35. Norfolk_terrier
36. Norwich_terrier
37. Yorkshire_terrier
38. wire-haired_fox_terrier
39. Lakeland_terrier
40. Sealyham_terrier
41. Airedale
42. cairn
43. Australian_terrier
44. Dandie_Dinmont
45. Boston_bull
46. miniature_schnauzer
47. giant_schnauzer
48. standard_schnauzer
49. Scotch_terrier
50. Tibetan_terrier
51. silky_terrier
52. soft-coated_wheaten_terrier
53. West_Highland_white_ter

## Project Progress Report

<div style="width: 100%; display: inline-block;">
    <img src="data/project_report.jpg" style="width: 38%; margin: 0px; display: inline; float: left;">
    <div style="width: 
60%; display: inline-block; float: right; font-size: 16px; padding-left: 10px">
        <p>
            The images in the dataset are of varying dimensions. But for the analysis, all the images are resized into <b>250 x 250 pixels</b>.<br><br>
            We are using color images for the analysis, so each pixel will itself conatins 3 values representing <b>(R, G, B)</b>.<br>
        </p>
    </div>
</div>

###  Design of the Convolutional Neural network

For the first iteration, we overtook the following design of the convolution neural network:

__
Input layer: (250, 250, 3)<br>
<br>
1st Convolution Layer: 32 filters of size 5x5<br>
1st Pooling Layer: 2x2 filter with a stride of 2<br>
<br>
2nd Convolution Layer: 32 filters of size 5x5<br>
<br>
3rd Convolution Layer: 32 filters of size 5x5<br>
2nd Pooling Layer: 2x2 filter with a stride of 2<br>
<br>
1st Dense Layer: 1024 neurons<br>
2nd Dense Layer: 120 neurons (One for each class)<br>
__

The <b>Convlutional Layer</b> is the core building block of a Convolutional Neural Network that does most of the computational heavy lifting.

The <b>Pooling Layer</b> is to progressively reduce the spatial size of the representation to reduce the amount of parameters and computation in the network, and hence to also control overfitting.

The <b>Dense Layer</b> comprises of the traditional neural network which classifies the input to their respective classes.

### Tools available for modelling the neural networks

The most common way to implement any neural network is to use pre-defined libraries instead of designing it from the scratch. The following are the list of libraries that we analysed for this use case:

* __Tensorflow__ - https://www.tensorflow.org/
* __Keras__ - https://keras.io/
* __Theano__ - http://deeplearning.net/software/theano/
* __CNTK__ - https://www.microsoft.com/en-us/cognitive-toolkit/

We decided to go with __Tensorflow__ due to its ease of use and high level tuning of hyper parameters.

#### We have implemented the first iteration of the model using tensorflow.

In [15]:
import sys
from os.path import expanduser
sys.path.append(expanduser("~") + "/.local/lib/python3.5/site-packages")

In [16]:
# %load cnn_classifier.py
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from os.path import join
from image_reader import read_image
from tqdm import tqdm

import deep_neural_network as network
import pandas as pd
import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)
READ_DATA_FROM_FILE = True


def main(unused_argv):
    data_dict = pd.read_csv('./data/data_dict.csv')
    # Loading the data
    train_data = data_dict[data_dict["type"] == "train"][["file_name", "breed_name"]].copy()
    test_data = data_dict[data_dict["type"] == "test"][["file_name", "breed_name"]].copy()
    labels = data_dict["breed_name"].unique()
    label_mapping = {}

    count = 0
    for label in labels:
        label_mapping[label] = count
        count += 1
    del count

    # Due to CPU and memory constraint using less number of training data
    train_data = train_data.sample(2000)
    test_data = test_data.sample(500)

    x_train = x_test = None
    y_train = y_test = []

    if READ_DATA_FROM_FILE:
        # Loading data from file
        x_train = np.load("data/train_data_x.npy")
        y_train = np.load("data/train_data_y.npy")
        x_test = np.load("data/test_data_x.npy")
        y_test = np.load("data/test_data_y.npy")
    else:
        with tqdm(total=train_data.shape[0], desc="Reading train data") as pbar:
            for row in train_data.itertuples():
                folder_name = row.file_name.split("_")[0] + "-" + row.breed_name
                img_data = read_image(join("./data", "Images", folder_name, "%s.jpg" % row.file_name), (100, 100))
                if x_train is None:
                    x_train = [img_data]
                else:
                    x_train = np.append(x_train, [img_data], axis=0)
                y_train = np.append(y_train, label_mapping[row.breed_name])
                pbar.update(1)

        with tqdm(total=test_data.shape[0], desc="Reading test data") as pbar:
            for row in test_data.itertuples():
                folder_name = row.file_name.split("_")[0] + "-" + row.breed_name
                img_data = read_image(join("./data", "Images", folder_name, "%s.jpg" % row.file_name), (100, 100))
                if x_test is None:
                    x_test = [img_data]
                else:
                    x_test = np.append(x_test, [img_data], axis=0)

                y_test = np.append(y_test, label_mapping[row.breed_name])
                pbar.update(1)

        # Writing the train and test data to a file
        np.save("data/train_data_x.npy", x_train)
        np.save("data/train_data_y.npy", y_train)
        np.save("data/test_data_x.npy", x_test)
        np.save("data/test_data_y.npy", y_test)

    # Creating estimator
    classifier = tf.estimator.Estimator(
        model_fn=network.create_cnn_model,
        model_dir="./tmp/model_data"
    )

    # Setting up logging for predictions
    tensors_to_log = {
        "probabilities": "softmax_tensor"
    }
    logging_hook = tf.train.LoggingTensorHook(
        tensors=tensors_to_log,
        every_n_iter=100
    )

    # Training the model
    print("Training the model...")
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": x_train},
        y=y_train,
        batch_size=100,
        num_epochs=None,
        shuffle=True)
    classifier.train(
        input_fn=train_input_fn,
        steps=2000,
        hooks=[logging_hook])

    # Evaluate the model and print results
    print("Evaluating the model...")
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": x_test},
        y=y_test,
        num_epochs=1,
        shuffle=False)
    eval_results = classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)


if __name__ == "__main__":
    tf.app.run()


Using TensorFlow backend.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': './tmp/model_data', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


INFO:tensorflow:Using config: {'_model_dir': './tmp/model_data', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


...

INFO:tensorflow:Saving dict for global step 4000: accuracy = 0.036, global_step = 4000, loss = 9.22628


INFO:tensorflow:Saving dict for global step 4000: accuracy = 0.036, global_step = 4000, loss = 9.22628


{'accuracy': 0.035999998, 'loss': 9.2262764, 'global_step': 4000}


SystemExit: 

/home/raghavan/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Issues faced when implementing the model

<p style="font-size: 16px">
    Due to the memory constraint, there was a trade-off between the quality of the image and the number of training sets. <br>
    We were able to load only 2000 - 5000 taining images with <b>100 x 100</b> resolution or only about 500 - 1000 training images with <b>250 x 250</b> resolution from a 12000 images dataset. <br>
    <br>
    Even though our model is very basic model with no tuning, this had a huge imapact towards the accuracy.<br>
    Accuracy obtained from this iteration: <b>3%</b>
<p>

<img src="data/ruff_time.jpg">